<h1 align='center' ><b><i> Crimes In Boston </b></i></h1>

<p align=center>
  <img src="https://live.staticflickr.com/65535/51728157715_ff5aa3f2e3_z.jpg" width="30%">
</p>

The Dataset used is from [Analyze Boston](https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system). It has the data from 15 June 2015 to 14 October 2024. In this notebook we will clean the dataset in order to analyze it.

### <span style='color: #339966'>**Import libraries**</span>

In [1]:
#Libraries to treat data
import numpy as np
import pandas as pd
import json
import difflib

#Don't show warning messages
import warnings

warnings.filterwarnings('ignore')

#Configure pandas options
pd.set_option('max_seq_items', None)
pd.set_option('max_colwidth', None)

### <span style='color:#339966'>**Data Import and Dictionary** </span>
The Data Dictionary can be found at this [link](https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system/resource/9c30453a-fefa-4fe0-b51a-5fc09b0f4655).

* **Incident_number:** Internal BPD number
* **Offense_code:** Numerical code of offense description
* **Offense_code_group:** Internal categorization of *Offense_code*
* **Offense_description:** Primary descriptor of incident
* **District:** What district the crime was reported in
* **Reporting_area:** RA number associated with where the crime was reported from
* **Shooting:** Indicated a shooting took place
* **Occurred_on_date:** Earliest date and time the incident could have taken place
* **Year:** The year that the crime occurred
* **Month:** The month that the crime occurred
* **Day_of_week:** The day of the week that the crime occurred
* **Hour:** The hour that the crime occurred
* **UCR_part:** Universal Crime Reporting Part number(1,2,3)
* **Street:** Street name the incident took place
* **Lat:** Latitude the incident took place
* **Long:** Longitude the incident took place
* **Location:** Location the incident took place



In [2]:
#import dataset
path = '../DataSets/crime_boston.csv'
boston_original = pd.read_csv(path)

#shows five first rows
boston_original.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I172040657,2629,Harassment,HARASSMENT,C11,397,NaN,2015-06-15 00:00:00,2015,6,Monday,0,Part Two,MELBOURNE ST,42.291093,-71.065945,"(42.29109287, -71.06594539)"
1,I182061268,3201,Property Lost,PROPERTY - LOST,NaN,,NaN,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,BERNARD,NaN,NaN,NaN
2,I162013546,3201,Property Lost,PROPERTY - LOST,B3,433,NaN,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,NORFOLK ST,42.283634,-71.082813,"(42.28363434, -71.08281320)"
3,I152051083,3115,Investigate Person,INVESTIGATE PERSON,A7,20,NaN,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,PARIS ST,42.377023,-71.032247,"(42.37702319, -71.03224730)"
4,I152059178,2647,Other,THREATS TO DO BODILY HARM,C11,359,NaN,2015-06-15 00:00:00,2015,6,Monday,0,Part Two,WASHINGTON ST,42.293606,-71.071887,"(42.29360585, -71.07188650)"


Let's create a copy of the original Dataset

In [3]:
#Create a copy of the dataset
boston = boston_original.copy()

#Dataset Shape
print("The Dataset Crimes In Boston Has {} lines and {} columns.".format(boston.shape[0], boston.shape[1]))

The Dataset Crimes In Boston Has 795933 lines and 17 columns.


### <span style='color: #339966'>**Dealing with missing Data**</span>
How much missing data we have in the DataFrame?

In [4]:
#Dataset missing values
print("What is the percentage of missing data per column?\n")
((boston.isnull().sum() / len(boston)) * 100).sort_values(ascending=False).round(2)

What is the percentage of missing data per column?



UCR_PART               55.63
OFFENSE_CODE_GROUP     55.62
SHOOTING               44.20
Location                5.56
Long                    5.56
Lat                     5.56
REPORTING_AREA          3.80
STREET                  1.49
DISTRICT                0.58
HOUR                    0.00
INCIDENT_NUMBER         0.00
DAY_OF_WEEK             0.00
MONTH                   0.00
OFFENSE_CODE            0.00
OCCURRED_ON_DATE        0.00
OFFENSE_DESCRIPTION     0.00
YEAR                    0.00
dtype: float64

As we can see above, we have a lot of missing data in some columns. The first thing we are going to do is to analyze and complete this data. Starting with the **SHOOTING** column.
Let's see what kind of data there is in this column.

#### <span style='color: #6fa832'>Shooting</span>

In [5]:
#Types of data in Shooting column
print("In the Shooting column, there are {} unique values that repeat in the entire dataset.".format(boston.SHOOTING.nunique()))
print(boston.SHOOTING.unique())
print("The type of data in the column: ", boston.SHOOTING.dtype)

In the Shooting column, there are 5 unique values that repeat in the entire dataset.
[nan 'Y' '0' '1' 0 1]
The type of data in the column:  object


The **SHOOTING** column has 5 different types of data, 3 object types, and 2 that should be integer types. When we load the data, the pandas library changes the 2 integers to object, so this way we have no conflict in manipulating the dataset.

After changing the "*NaN*" for 0, which indicates that there was no shooting, we are going to replace the object data type to integer type and put all in the same pattern, 0 = No Shooting, 1 = Shooting.

First we import the Shooting dataset, it also cames from [Analyse Boston](https://data.boston.gov/dataset/shootings), this dataset starts its records on 2015-01-01, and the our dataset starts in 2015-06-15, so we get only the lines that was inputed after this data. For start we verify which occurences happens in both datasets, in this case we're going to put the SHOOTING column in the dataset = 1. We start verifyng the non-missing values.

The second step is to verify the lines with missing value in SHOOTING column, if it is in the shooting dataset, we put = 1 as well.

The rest of the missing values, the ones that has no correspondent in the shooting dataset, we replace for 0 value.

In [6]:
#Fill missing data Shooting column with 0
boston["SHOOTING"].fillna(0, inplace=True)

#Change shooting values to 0 or 1
boston["SHOOTING"].replace(["Y", "1"], 1, inplace=True)
boston["SHOOTING"].replace("0", 0, inplace=True)

print("Missing values on Shooting column now: {}".format(boston["SHOOTING"].isnull().sum()))
print("Values in the column: {}".format(boston["SHOOTING"].unique()))

print("Data type of the column: {}".format(boston["SHOOTING"].dtype))

Missing values on Shooting column now: 0
Values in the column: [0 1]
Data type of the column: int64


In [7]:
#Load the shooting dataset
path_shoot = "../DataSets/Shootings.csv"
shooting_original = pd.read_csv(path_shoot)
shooting_original.head()

,incident_num,shooting_date,district,shooting_type_v2,victim_gender,victim_race,victim_ethnicity_nibrs,multi_victim
0,I152000196-00,2015-01-01 13:30:00+00,C11,Non-Fatal,Male,Black or African American,Not Hispanic or Latinx,f
1,I152000879-00,2015-01-04 01:58:00+00,E18,Non-Fatal,Male,Black or African American,Not Hispanic or Latinx,f
2,I152001014-00,2015-01-04 18:52:00+00,E5,Non-Fatal,Male,Black or African American,Not Hispanic or Latinx,f
3,I152002232-00,2015-01-08 22:45:00+00,B2,Non-Fatal,Male,Black or African American,Not Hispanic or Latinx,t
4,I152002233-00,2015-01-08 22:45:00+00,B2,Non-Fatal,Female,Black or African American,Not Hispanic or Latinx,t


In [8]:
#Shows the start date of the Dataset
print(f"Our Dataset starts on date {boston.OCCURRED_ON_DATE.min()}.")

Our Dataset starts on date 2015-06-15 00:00:00.


In [9]:
#Select the lines that is compatible with the dataset data
shooting = shooting_original.query("shooting_date >= '2015-06-15'")

In [10]:
#Shooting dataset shape
print(f"The shooting dataset has {shooting.shape[0]} lines and {shooting.shape[1]} columns.")

The shooting dataset has 1937 lines and 8 columns.


We have addressed the issue of the '+00' at the end of some rows in the 'shooting_date' and 'OCCURRED_ON_DATE' columns in the Shooting and Crime datasets respectively, which made it difficult to analyze and combine them. The date format has been standardized after removing the '+00'.

In [11]:
#Changing the data format on both datasets
boston['OCCURRED_ON_DATE'] = boston['OCCURRED_ON_DATE'].apply(lambda x: x.split('+')[0])
shooting['shooting_date'] = shooting['shooting_date'].apply(lambda x: x.split('+')[0])

In [12]:
#Check the changes
shooting.tail()

,incident_num,shooting_date,district,shooting_type_v2,victim_gender,victim_race,victim_ethnicity_nibrs,multi_victim
2021,242083083,2024-09-29 01:05:00,E18,Fatal,Male,Black or African American,Not Hispanic or Latinx,f
2022,242082507,2024-09-27 11:16:00,B3,Non-Fatal,Male,Unknown,Unknown,f
2023,242085392,2024-10-06 13:16:00,B3,Non-Fatal,Male,Black or African American,Unknown,f
2024,242085491,2024-10-06 19:54:00,C11,Non-Fatal,Unknown,Unknown,Unknown,f
2025,242085815,2024-10-08 01:35:00,D4,Fatal,Unknown,Unknown,Unknown,f


In addition, there are two incident numbers that are different from the others, we put them in the same way.

In [13]:
#Some of the incident numbers that where not in the standard
shooting.loc[180, 'incident_num'] = 'I152078475'
shooting.loc[1098, 'incident_num'] = 'I192084830'
# shooting.loc[1363, 'incident_num'] = '200559783'

We ensured that there were no missing values in the shooting column of the crime data by cross-referencing it with the shooting dataset.

In [14]:
#Get the index of date and incident_number in shooting dataset
date = shooting['shooting_date'].unique() #Get all the unique date of shooting dataframe
incident_number = shooting['incident_num'].unique() #Get all the unique incident numbers of shooting dataframe

#Get all the index in the boston dataset that has a correspondent in the shooting dataset
index_boston_shooting = boston.loc[(boston['INCIDENT_NUMBER'].isin(shooting['incident_num'])) |
                                   (boston['OCCURRED_ON_DATE'].isin(shooting['shooting_date']))].index

count_0 = 0 #get the amount of lines that has shooting =0 but it is in the other dataset
count = 0

#Verify which lines that are in the boston dataset and in the shooting dataset at the same time
for i in index_boston_shooting:
    if boston.loc[i, 'OCCURRED_ON_DATE'] in date:
        aux_date = boston.loc[i, 'OCCURRED_ON_DATE'] #Get the date from the line
        if (boston.loc[i, 'DISTRICT'] == shooting[shooting['shooting_date']==aux_date]['district'].unique()):
            if boston.loc[i,'SHOOTING'] == 0:
                count_0+=1

            boston.loc[i, 'SHOOTING'] = 1 #Replace the value for 1
            count+=1 #Calculate the lines that correspond in the second dataset

    if boston.loc[i, 'INCIDENT_NUMBER'] in incident_number:
        boston.loc[i, 'SHOOTING'] = 1
        count+=1


print("Amount of index shooting that has a correspondent in the shooting dataset: ",count)
print(f"There was {count_0} lines with 0 value that it is in the Shooting dataset, so we set as 1 too.")

Amount of index shooting that has a correspondent in the shooting dataset:  3164
There was 108 lines with 0 value that it is in the Shooting dataset, so we set as 1 too.


The Shooting dataset contains some rows that are not present in the Crimes dataset. However, it's important to note that the latter is only a sample of the crimes that have occurred in Boston since 2015.

In [15]:
#Get all lines that are not in boston dataframe
not_in_boston = shooting.loc[~((shooting['incident_num'].isin(boston['INCIDENT_NUMBER'])) |
                               (shooting['shooting_date'].isin(boston['OCCURRED_ON_DATE'])))]

print(f"There are {len(not_in_boston)} lines that are not in the Boston Dataset.")

There are 242 lines that are not in the Boston Dataset.


#### <span style='color: #6fa832'>UCR</span>

Let's now turn our attention to the UCR, or Uniform Crime Report, which is the second column we will be discussing. This program was created by the FBI with the aim of gathering data on crimes throughout the United States. While the FBI divides the UCR into two parts, the Data Frame divides it into four.

*  **Part 1** is related to serious crimes, with the main offenses of this group being: *criminal homicide*, *forcible rape*, *robbery*, *burglary*, *larceny-theft*, *motor vehicle theft*, *arson*, *human trafficking*, and *involuntary servitude*.

* Additionally, **Part 2** categorizes other crimes, including *simple assault*, *fraud*, *vandalism*, *sex offenses*, *drug abuse violations*, and more.

* Finally, **Part 3** and Other encompass crimes of lesser severity. The FBI does not use that classification, but we will keep it as is since it appears in these Data Frames.

Let's see the amount of each Part in the column.



In [16]:
#Before
print("Unique values in the column: ", boston.UCR_PART.unique(), "\n")
print("Before filling the column, these are the values amount:\n")
print(boston.UCR_PART.value_counts())
print("nan: ", boston.UCR_PART.isna().sum())

Unique values in the column:  ['Part Two' 'Part Three' 'Part One' 'Other' nan] 

Before filling the column, these are the values amount:

Part Three    176042
Part Two      107656
Part One       68068
Other           1390
Name: UCR_PART, dtype: int64
nan:  442777


To get started, we'll create three arrays that contain the offense codes for the specific UCR parts. These arrays will be used to replace any lines that don't have a UCR code. With this approach, we can ensure that all offenses are properly categorized and accounted for.

In [17]:
#create offense arrays per part
offense_part1 = boston.OFFENSE_CODE.loc[boston["UCR_PART"] == "Part One"].unique()
offense_part2 = boston.OFFENSE_CODE.loc[boston["UCR_PART"] == "Part Two"].unique()
offense_part3 = boston.OFFENSE_CODE.loc[boston["UCR_PART"] == "Part Three"].unique()

#Fill all na for "Other" value
boston.UCR_PART.fillna("NaN", inplace = True)

#Replace the NaN value according to Offense code
boston.loc[boston.OFFENSE_CODE.isin(offense_part1), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part One")
boston.loc[boston.OFFENSE_CODE.isin(offense_part2), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part Two")
boston.loc[boston.OFFENSE_CODE.isin(offense_part3), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part Three")

In [18]:
#After
print("After filling the column:")
boston.UCR_PART.value_counts()

After filling the column:


Part Three    393877
Part Two      234242
Part One      151625
NaN            14799
Other           1390
Name: UCR_PART, dtype: int64

We can easily reclassify these values by comparing them with those classified as 'Other' and cross-referencing them with the code on the [Analyze Boston website](https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system/resource/3aeccf51-a231-4555-ba21-74572b4c33d6) and the official [FBI site](https://www2.fbi.gov/ucr/cius_04/appendices/appendix_02.html). This approach will help us to better understand the data.

*Others:*
<table>
    <tr><th> 735 </th> <td> Recovered - MV Recovered In Boston/Stolen Outside Boston </td> <th> Part Three </th></tr>
    <tr><th> 670 </th> <td> Recovered Stolen Plate </td> <th> Part Three </th></tr>
    <tr><th> 900 </th> <td> Arson </td> <th> Part One </th></tr>
    <tr><th> 2631 </th> <td> Property - Concealing Leased </td> <th> Part Three </th></tr>
    <tr><th> 547 </th> <td> B&E Non-Residence Day - No Prop Taken </td> <th> Part One </th></tr>
    <tr><th> 121 </th> <td> Manslaughter - Vehicle - Negligence </td> <th> Part Three </th></tr>
    <tr><th> 123 </th> <td> Manslaughter - Non-Vehicle - Negligence </td> <th> Part One </th></tr>
    <tr><th> 112 </th> <td> Killing Of Fellon By Police </td> <th> Part One </th></tr>
    <tr><th> 527 </th> <td> B&E Residence Day - No Prop Taken </td> <th> Part One </th></tr>
</table>

*NaN:*
<table>
    <tr><th> 2010 </th> <td> Home Invasion </td> <th> Part One </th></tr>
    <tr><th> 1610 </th> <td> Human Traficking - Commercial Sex Acts </td> <th> Part One </th></tr>
    <tr><th> 1620 </th> <td> Human Traficking - Involuntary Servitude </td> <th> Part One </th></tr>  
    <tr><th> 531 </th> <td> B&E Non-Residence Night - Attempt Force </td> <th> Part One </th></tr>
    <tr><th> 3300 </th> <td> Migrated Report - Other </td> <th> Part Three </th></tr>
    <tr><th> 700 </th> <td> Migrated Report - Auto Theft </td> <th> Other </th></tr>
    <tr><th> 3000 </th> <td> Migrated Report - Death Investigation </td> <th> Part One </th></tr>
    <tr><th> 2500 </th> <td> Migrated Report - Kidnapping </td> <th> Other </th></tr>
    <tr><th> 400 </th> <td> Migrated Report - Aggravated Assault/Agrravated Assault & Battery </td> <th> Part One </th></tr>
    <tr><th> 800 </th> <td> Migrated Report - Assault/Assault & Battery </td> <th> Part Two </th></tr>
    <tr><th> 1100 </th> <td> Migrated Report - Fraud </td> <th> Part Two </th></tr>
    <tr><th> 736 </th> <td> Recovered - MV Recovered In Boston (Stolen In Boston) </td> <th> Part Three </th></tr>
    <tr><th> 641 </th> <td> Breaking And Entering (B&E) Motor Vehicle </td> <th> Part One </th></tr>
    <tr><th> 2950 </th> <td> Migrated Report - Motor Vehicle Crash </td> <th> Other </th></tr>
    <tr><th> 1400 </th> <td> Migrated Report - Vandalism/Destruction of Property </td> <th> Part Two </th></tr>
    <tr><th> 600 </th> <td> Migrated Report -  Other Larceny </td> <th> Part One </th></tr>
    <tr><th> 3200 </th> <td> Migrated Report - Investigate Person </td> <th> Other </th></tr>
    <tr><th> 2600 </th> <td> Migrated Report - Other Part II </td> <th> Part Two </th></tr>
    <tr><th> 2400 </th> <td> Migrated Report - Affray/Disturbing The Peace/Disorderly Conduct </td> <th> Other </th></tr>
    <tr><th> 500 </th> <td> Migrated Report - Burglary/Breaking and Entering </td> <th> Part One </th></tr>
    <tr><th> 650 </th> <td> Migrated Report - Larceny From MV </td> <th> Part Two </th></tr>
    <tr><th> 1800 </th> <td> Migrated Report - Drugs - Possession/Manufacturing/Distribute </td> <th> Part Two </th></tr>
    <tr><th> 3100 </th> <td> Migrated Report - Investigate Property </td> <th> Other </th></tr>
    <tr><th> 300 </th> <td> Migrated Report - Robbery </td> <th> Part Two </th></tr>
    <tr><th> 1000 </th> <td> Migrated Report - Counterfeiting/Forgery </td> <th> Part Two </th></tr>
    <tr><th> 1500 </th> <td> Migrated Report - Weapons Violation </td> <th> Part Two </th></tr>
    <tr><th> 3350 </th> <td> Migrated Report - Injured/Medical/Sick Assist </td> <th> Other </th></tr>
    <tr><th> 100 </th> <td> Migrated Report - Criminal Homicide </td> <th> Part One </th></tr>
    <tr><th> 750 </th> <td> Migrated Report - Other Auto Theft </td> <th> Part Two </th></tr>
    <tr><th> 1200 </th> <td> Migrated Report - Embezzlement </td> <th> Part Two </th></tr>
    <tr><th> 3005 </th> <td> Sick Assist </td> <th> Other </th></tr>
    <tr><th> 3126 </th> <td> Warrant Arrest - Outside Of Boston Warrant </td> <th> Other </th></tr>
    <tr><th> 2671 </th> <td> Violation - Harassment Prevention Order </td> <th> Other </th></tr>
    <tr><th> 99999 </th> <td> Evidence Tracker Incidents </td> <th> Other </th></tr>
    <tr><th> 990 </th> <td> Justificable Homicide </td> <th> Part One </th></tr>
</table>

The Offense Codes **1610** and **1620**, according to this [document](https://ucr.fbi.gov/human-trafficking), are classified as Part One.



In [19]:
#Verify Others and NaN
print("Offense Code classify as Others:")
print(boston.OFFENSE_CODE.loc[boston.UCR_PART == "Other"].unique(), "\n")

print("Offense Code classify as NaN:")
print(boston.OFFENSE_CODE.loc[boston.UCR_PART == "NaN"].unique())

Offense Code classify as Others:
[ 735  670  900 2631  547  527  121  112  123] 

Offense Code classify as NaN:
[ 2010  1620  1610   735   600   400   100  2631   800  3000   900   500
  1800  1100   670   700  1000  3300  1500  3200  2600   121   300  3100
  1400  1200  2950   650  2400   641  3350   736  2500  3005   531  3126
  2671   990 99999   122]


In [20]:
#create a list with the codes above according to UCR
part_one = [900, 547, 123, 112, 527, 2010, 1610, 1620, 531, 3000, 400, 641, 600, 500, 100, 990, 122]
part_two = [800, 1100, 1400, 650, 1800, 300, 1000, 1500, 750, 1200]
part_three = [735, 670, 2631, 121, 3300, 736]
other = [700, 2500, 2950, 3200, 2600, 2400, 3100, 3350, 3005, 3126, 2671, 99999, 990]

#Replace Others and NaN values in UCR column.
boston.loc[boston.OFFENSE_CODE.isin(part_one), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part One")
boston.loc[boston.OFFENSE_CODE.isin(part_one), "UCR_PART"]= boston.UCR_PART.str.replace("Other", "Part One")
boston.loc[boston.OFFENSE_CODE.isin(part_two), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part Two")
boston.loc[boston.OFFENSE_CODE.isin(part_two), "UCR_PART"] = boston.UCR_PART.str.replace("Other", "Part Two")
boston.loc[boston.OFFENSE_CODE.isin(part_three), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Part Three")
boston.loc[boston.OFFENSE_CODE.isin(part_three), "UCR_PART"] = boston.UCR_PART.str.replace("Other", "Part Three")
boston.loc[boston.OFFENSE_CODE.isin(other), "UCR_PART"] = boston.UCR_PART.str.replace("NaN", "Other")

In [21]:
#Final count
print("The final count of unique UCR values:")
boston.UCR_PART.value_counts()

The final count of unique UCR values:


Part Three    396778
Part Two      234393
Part One      152334
Other          12428
Name: UCR_PART, dtype: int64

#### <span style='color: #6fa832'>Offense_code_group</span>
Here we start with all columns offense_code_group, offense_description and street with only the first letters of each word capitalized.

In [22]:
#Changes the writing to Title style
index_isna = boston[~boston['OFFENSE_CODE_GROUP'].isna()]['OFFENSE_CODE_GROUP'].index #offense_code_group not null
index_isna_street = boston[~boston['STREET'].isna()]['STREET'].index #street not null

boston.loc[index_isna, 'OFFENSE_CODE_GROUP'] = boston.loc[index_isna, 'OFFENSE_CODE_GROUP'].apply(lambda x: x.title())
boston['OFFENSE_DESCRIPTION'] = boston['OFFENSE_DESCRIPTION'].apply(lambda x: x.title())
boston.loc[index_isna_street, 'STREET'] = boston.loc[index_isna_street, 'STREET'].apply(lambda x: x.title())

We will create a dictionary with offense_code as key and offense_code_group as value. This will be used to fill in the missing values in the offense_code_group column.

In [23]:
#Create a dictionary with the offense_code and unique offense_code_group
offense_code = boston.groupby('OFFENSE_CODE')['OFFENSE_CODE_GROUP'].unique().apply(lambda x: x[0]).to_dict()
print(offense_code)

{100: nan, 111: 'Homicide', 112: 'Manslaughter', 121: 'Manslaughter', 122: nan, 123: 'Manslaughter', 300: nan, 301: 'Robbery', 311: 'Robbery', 315: 'Robbery', 334: 'Robbery', 335: 'Robbery', 338: 'Robbery', 339: 'Robbery', 349: 'Robbery', 351: 'Robbery', 361: 'Robbery', 371: 'Robbery', 381: 'Robbery', 400: nan, 402: 'Aggravated Assault', 403: 'Aggravated Assault', 404: 'Aggravated Assault', 413: 'Aggravated Assault', 423: 'Aggravated Assault', 432: 'Aggravated Assault', 500: nan, 511: 'Residential Burglary', 520: 'Residential Burglary', 521: 'Residential Burglary', 522: 'Residential Burglary', 527: 'Burglary - No Property Taken', 530: 'Commercial Burglary', 531: nan, 540: 'Commercial Burglary', 541: 'Commercial Burglary', 542: 'Commercial Burglary', 547: 'Burglary - No Property Taken', 560: 'Other Burglary', 561: 'Other Burglary', 562: 'Other Burglary', 600: nan, 611: 'Larceny', 612: 'Larceny', 613: 'Larceny', 614: 'Larceny From Motor Vehicle', 615: 'Larceny From Motor Vehicle', 616: '

In [24]:
#Changes some of the NaN values to correspondent in the dictionary created above
boston['OFFENSE_CODE_GROUP'] = boston['OFFENSE_CODE'].map(offense_code)

Now we will create a list of all offense_codes that do not have an offense_code_group.

In [25]:
#Separate all offese_code_group that is nan
offense_nan = []
for offense in offense_code:
    if  pd.isna(offense_code[offense]):
        offense_nan.append(offense)

print(offense_nan)

[100, 122, 300, 400, 500, 531, 600, 641, 650, 700, 736, 800, 990, 1000, 1100, 1200, 1400, 1500, 1800, 2400, 2500, 2600, 2671, 2950, 3000, 3005, 3100, 3126, 3200, 3300, 3350, 99999]


In [26]:
#Check if there is any other value for offense_code_group for the offense code we've separetad before
boston.loc[boston['OFFENSE_CODE'].isin(offense_nan)]['OFFENSE_CODE_GROUP'].unique()

array([nan], dtype=object)

Looking at the dataset, we conclude that we can use the offense_description column to fill in the missing values that we still have in offense_code. 

In [27]:
#Put in a dictionary all the missing values that we still have, this time we use the offense_description to fill in
nan_dict = boston.loc[boston['OFFENSE_CODE'].isin(offense_nan)].groupby("OFFENSE_CODE")\
    ['OFFENSE_DESCRIPTION'].unique().apply(lambda x: x[0].split('-')[0]).apply(lambda x: x.split('(')[0]).to_dict()

print(nan_dict)

{100: 'Migrated Report ', 122: 'Manslaughter ', 300: 'Migrated Report ', 400: 'Migrated Report ', 500: 'Migrated Report ', 531: 'Breaking And Entering ', 600: 'Migrated Report ', 641: 'Breaking And Entering ', 650: 'Migrated Report ', 700: 'Migrated Report ', 736: 'Recovered ', 800: 'Migrated Report ', 990: 'Justifiable Homicide', 1000: 'Migrated Report ', 1100: 'Migrated Report ', 1200: 'Migrated Report ', 1400: 'Migrated Report ', 1500: 'Migrated Report ', 1800: 'Migrated Report ', 2400: 'Migrated Report ', 2500: 'Migrated Report ', 2600: 'Migrated Report ', 2671: 'Violation ', 2950: 'Migrated Report ', 3000: 'Migrated Report ', 3005: 'Sick Assist', 3100: 'Migrated Report ', 3126: 'Warrant Arrest ', 3200: 'Migrated Report ', 3300: 'Migrated Report ', 3350: 'Migrated Report ', 99999: 'Evidence Tracker Incidents'}


In [28]:
#Get the indexes of missin offense_code_group e fill according the dict we create above
nan_ind = boston.loc[boston['OFFENSE_CODE'].isin(nan_dict.keys())].index
boston.loc[nan_ind, 'OFFENSE_CODE_GROUP'] = boston.loc[nan_ind, 'OFFENSE_CODE'].map(nan_dict)

In [29]:
#Final result
print(f"We had {boston_original['OFFENSE_CODE_GROUP'].isna().sum()} missing values before, after \
filling with the above methods we have {boston['OFFENSE_CODE_GROUP'].isna().sum()}.")

We had 442680 missing values before, after filling with the above methods we have 0.


We still have to deal with the STREET, Lat, Long and Location columns, which have missing values. If we look at the STREET column, there are several different entries for the same street, so we will create a new data frame with the unique entries for Location, Lat and Long, and we will use this information in this [python script](https://github.com/DenisePotenza/Portfolio/blob/main/Crimes_In_Boston/Python_Code/geolocation_boston.py) to get the correct street name for each location.

In [30]:
#Show all lines that the location is diferent
boston[boston['Location'].str.contains('e') == True]

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
510418,202094496,3802,Motor Vehicle Accident Response,M/V Accident - Property Damage,B2,,0,2020-12-27 03:47:00,2020,12,Sunday,3,Part Three,Gayland & W Cottage St\nRoxbury Ma 02119\nUnited St,1.327276e-07,5.249691e-08,"(1.32727612912185e-07, 5.24969124614893e-08)"


In [31]:
#Changes the location for a valid location and drop the other line
boston.loc[510418,['Lat', 'Long', 'Location']] = 42.320483, -71.076698, '(42.320483, -71.076698)'

If you look closely, it's pretty clear that some locations have different numbers at the end of the latitude and longitude, but lead to exactly the same place. Using Google Maps, these places are shown as the same. Also, in Google Maps, the latitude and longitude have only 6 numbers after the floating point, so we will leave the location with the same amount of numbers.

In [32]:
#Reduce the latitude and longitude numbers
def short_loc(loc:str) -> str:
    lat, long = loc.replace('(', '').replace(')', '').replace(' ', '').split(',')
    return f'({float(lat):.6f},{float(long):.6f})'

In [33]:
#Applies the function we've created above to the Location column
ind_loc = boston[~boston['Location'].isna()].index
boston.loc[ind_loc, 'Location'] = boston.loc[ind_loc, 'Location'].apply(short_loc)

#Creates a DataFrame with the location, latitude and longitude that are uniques in the Dataset
location = boston[~boston['Location'].isna()]['Location'].unique()
loc_lat_long = pd.DataFrame(location, columns=['Location'])
loc_lat_long['Lat'] = loc_lat_long['Location'].apply(lambda x: x.replace('(', '').replace(')', '').split(',')[0])
loc_lat_long['Long'] = loc_lat_long['Location'].apply(lambda x: x.replace('(', '').replace(')', '').split(',')[1])

In [34]:
#Saves the dataFrame we've created to an csv file
loc_lat_long.to_csv('../DataSets/loc_lat_long.csv', index=False)

With the dataset filled with the correct street name and suburb by geopy, we will use it to change the street names in the crime dataset and will create a new column with the suburb.

In [35]:
#Get the DataFrame with the streets and Suburbs names
path_lat_long = "../DataSets/loc_lat_long_suburb.csv"
street_suburb = pd.read_csv(path_lat_long)
street_suburb.head()

,Location,Lat,Long,Street,Suburb
0,"(42.291093,-71.065945)",42.291093,-71.065945,Melbourne Street,Dorchester
1,"(42.283634,-71.082813)",42.283634,-71.082813,Norfolk Street,Dorchester
2,"(42.377023,-71.032247)",42.377023,-71.032247,Paris Street,East Boston
3,"(42.293606,-71.071887)",42.293606,-71.071887,Washington Street,Dorchester
4,"(42.328663,-71.085634)",42.328663,-71.085634,Washington Street,Roxbury


Using the dataset above, we will create 2 dictionaries, one for street names and the other for suburbs, both using location as the key.

In [36]:
#Creates 2 dictionarys, on containing the street names for each location, and the other the suburb names,
#this will be used to fill the Street column and create a new one with the Suburb
streets = pd.Series(street_suburb['Street'].values, index=street_suburb['Location']).to_dict()
suburbs = pd.Series(street_suburb['Suburb'].values, index=street_suburb['Location']).to_dict()

In [37]:
#Map the streets and create a new column for the suburbs by the location
boston.loc[~boston['Location'].isna(), 'STREET'] = boston['Location'].map(streets)
boston.loc[~boston['Location'].isna(), 'Suburb'] = boston['Location'].map(suburbs)

boston.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,Suburb
0,I172040657,2629,Harassment,Harassment,C11,397,0,2015-06-15 00:00:00,2015,6,Monday,0,Part Two,Melbourne Street,42.291093,-71.065945,"(42.291093,-71.065945)",Dorchester
1,I182061268,3201,Property Lost,Property - Lost,NaN,,0,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,Bernard,NaN,NaN,NaN,NaN
2,I162013546,3201,Property Lost,Property - Lost,B3,433,0,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,Norfolk Street,42.283634,-71.082813,"(42.283634,-71.082813)",Dorchester
3,I152051083,3115,Investigate Person,Investigate Person,A7,20,0,2015-06-15 00:00:00,2015,6,Monday,0,Part Three,Paris Street,42.377023,-71.032247,"(42.377023,-71.032247)",East Boston
4,I152059178,2647,Other,Threats To Do Bodily Harm,C11,359,0,2015-06-15 00:00:00,2015,6,Monday,0,Part Two,Washington Street,42.293606,-71.071887,"(42.293606,-71.071887)",Dorchester


#### <span style='color: #6fa832'>Separate the streets that has missing location</span>

In [38]:
#Get the name of the streets that has no location and remove the missing values
missing_location = list(boston[boston['Location'].isna()]['STREET'].unique())
missing_location = [value.title() for value in missing_location if type(value) == str]

print(f"We have, {len(missing_location)} street names that have no location.")

We have, 5305 street names that have no location.


In [39]:
#All the streets that has no location
print(missing_location)

['Bernard', 'Boylston St', 'Newcomb', 'Columbus Ave', 'Financial Ctr', 'Dorchester Ave', 'Glendale St', 'Hamilton', 'Day', 'Rowes', 'Charles', 'Allstate Rd', 'Centre St', 'Tara Dr', 'Tara Rd', 'Edgewater Rd', 'N Point Dr', 'Winslow Ave', 'Stuart St', 'Bussey St', 'Pitsmoor Rd', 'Gen', 'Humboldt Ave', 'Conley St', 'Columbia Rd', "O'Meara", 'Washington', 'Lorna', 'Massachusetts Ave', 'Windslow', 'Florida St', 'Hartford St', 'Fawndale', 'Dudley St', 'Elm Hill Ave', 'Boston Medical Center Pl', 'Marvin', 'Locust St', 'Appleton St', 'Orton Marotta', 'Long', 'Harbor', 'Public Alley No 442', '61 Hillside St', 'Harrison Ave', 'Clevemont Ave', 'Joy St', 'Nixon', "Monsignor Dennis F O'Call", 'King St', 'Bi Hosp', 'Washington St', 'John Eliot', 'Hyde', 'Cobden', 'Bragdon St', 'Evans St', 'North St', 'Albany St', 'Clarendon St', 'Arlington St', 'Mass', 'Franklin Field', 'Bowker St', 'Hanover St', 'Ruthven St', 'Pope John Paul Park', 'Storrow Dr', 'S Point Dr', '98 Woodledge St', 'School St', 'W Bou

We will create a dictionary to store the correct names of the streets.

In [40]:
# Correct dictionary
correct = {} 

#### <span style='color: #6fa832'> Functions

We will create some functions that will be used in the next steps: 
* ***fill_json***: takes a dictionary and uses it to update a json file with correct street names.
* ***remove_abb***: takes a list of street names and replaces the abbreviation with the full word, e.g. 'St' for 'Street'. Returns a dictionary with the street name in the dataset as the key and the changes as the value.
* ***compare_streetbook***: takes a list of street names and compares them with the street names in Boston and some other cities. To do this, we'll import a txt file called streetbook. The function returns a list of streets that are not in the streetbook file.
* ***matches_search***: takes a list of street names and uses difflib to see if there's a street that matches the streetbook but is misspelled or somehow incomplete. The streets that have a match with the streetbook are added to the correct dictionary and the same is removed from the list. It returns the same list without the matches.
* ***read_json***: takes a json file path and load.

In [41]:
#Function that retrieves a json archive and updates it with the correct dictionary
def fill_json (dict1: dict): #Open and update file
    with open('../Json_Files/correct_street.json', 'r') as json_file:
        file = json.load(json_file)
        file.update(dict1)

    file = dict(sorted(file.items()))
    
    with open('../Json_Files/correct_street.json', 'w') as sjson_file: #Save the updated file
        json.dump(file, sjson_file)

    json_file.close()

In [42]:
#Enter the names of the streets in the pattern used
def remove_abb(list_to_change: list) -> dict:
    # Dictionary of abbreviations and their replacements
    replacements = {
        'St': 'Street',
        'Ave': 'Avenue',
        'Av': 'Avenue',
        'Rd': 'Road',
        'Dr': 'Drive',
        'Ctr': 'Center',
        'Ln': 'Lane',
        'Sq': 'Square',
        'Wy': 'Way',
        'Blvd': 'Boulevard',
        'Blv': 'Boulevard',
        'Bl': 'Boulevard',
        'Pub': 'Public',
        'Hwy': 'Highway',
        'Pl': 'Place',
        'Plz': 'Plaza',
        'Visc': 'Visconti',
        'Whrf': 'Wharf',
        'Pa': 'Park',
        'Pk': 'Park',
        'Tr': 'Terrace',
        'Ter': 'Terrace',
        'Cirt': 'Circuit',
        'Pkwy': 'Parkway',
        'Mt': 'Mount',
        'Ct': 'Court',
        'S': 'South',
        'N': 'North',
        'W': 'West',
        'E': 'East'
    }
    
    dict_clean = {}
    
    for st in list_to_change:
        # Remove periods and split the street name into parts
        st_replace = st.replace('.', '')
        spl = st_replace.split(' ')
        
        # Iterate through each word and apply replacement logic
        for i, word in enumerate(spl):
            # If the word is "St" and it's the first word, replace with "Saint"
            if word == "St" and i == 0:
                spl[i] = "Saint"
            elif word == 'Dr' and i == 0:
                spl[i] = "Doctor"
            else:
                # Otherwise, replace based on the dictionary if the word is in it
                spl[i] = replacements.get(word, word)
        
        # Join the words back into a single string
        corr_street = ' '.join(spl)

        # Add the cleaned street name to the dictionary
        dict_clean[st] = corr_street


    return dict_clean

The streetbook file was obtained from the boston.gov website, it is a pdf file that can be found in this [link](https://www.boston.gov/sites/default/files/embed/s/streetbook_04262016.pdf). We remove the information using the [code](/home/denisepotenza/Projects/Crimes_In_Boston/Python_Code/boston_streetbook.py). During the process of cleaning the dataset I realised that some of the streets that appear were not in the streetbook file, so I added them manually. This is why the streetbook file is different from the pdf file.

In [43]:
#Open Streetbook txt file
file = open('../Txt/streetbook.txt', 'r')
streetbook_raw = file.readlines()
streetbook = [line.replace(',\n', '').replace('\n', '') for line in streetbook_raw]
file.close()

In [44]:
#Compare street names with streetbook file
def compare_streetbook(streets: list, streetbook=streetbook):

    """Checks if the street is in the Boston Streetbook text file."""

    streets[:] = [st for st in streets if st not in streetbook]

    return streets

In [45]:
#Search for matches in streetbook
def matches_search(matches: list):
    for match in matches.copy():
        if difflib.get_close_matches(match, streetbook, cutoff=0.8):
            correct[match] = difflib.get_close_matches(match, streetbook, cutoff=0.8)[0]
            matches.remove(match)
    return matches

In [46]:
#Import json files to cleanse the dataset
def read_json(json_file:json) -> json:
    with open(json_file, 'r') as file:
        j_file = json.load(file)

    file.close()
    return j_file

Using the compare_streetbook function, we'll check the missing_location to see which streets already exist in the streetbook file.

In [47]:
#Check if a street is in the Boston Streetbook and remove it from the list
missing_location = compare_streetbook(missing_location)

#### <span style='color: #6fa832'> Only_one and two_streets

The missing_location variable can be split into 2 other variables. One that contains the crossroads, so that we can use it to find the exact location, and the other that just contains the name of a street.

In [48]:
#Split the streets if there is a crossroads or just one street
only_one = [] #Gets the streets that has just one street
two_streets = [] #Gets crossroads

for street in missing_location:
    spl = street.split(' ')
    if ('And' in spl) or ('&' in spl) or ('@' in spl) or ("/" in spl) or ("At" in spl):
        two_streets.append(street)
    else:
        only_one.append(street)

There are some entries in the Street column that have the name of a police district, we will replace them with the correct address.

In [49]:
#Show in the not_similar list the Streets that has a district as a ???
dist = ['A1', 'A15', 'A7', 'B2', 'B3', 'C6', 'C11', 'D4', 'D14', 'E5', 'E13', 'E18', 'Dist']
districts = []
for d in dist:
    districts = districts + [s for s in only_one if d in s]

print(districts)

['B3', 'C6', 'Dist C11', 'D14', 'E5', 'E13', 'E18', 'Dist 18', 'Dist 7', 'District Wide', 'Dist 11', 'Dist 6', 'District Ave', 'Dist 5', 'Dist 13', 'Dist Wide', 'Dist 1', 'Dist13', 'Dist 14\nBrighton  Ma 02135\nUnited States', 'Nike District Ave', 'Dist C11']


In [50]:
#Districts to set as Unknown
dist_off = ['District Wide', 'District Ave', 'Dist Wide', 'District Avenue', 'Nike District Ave']
district_adress = list(set(districts).difference(dist_off))

#Remove districts from only_one
only_one = [st for st in only_one if st not in district_adress]

There are some streets that have no complement, so it is hard to say if it is an avenue, a street, a road, etc. We'll separate these streets in a variable called no_complement.

In [51]:
#Get all the streets that have only the name, without specifying that it is a street, road, avenue, etc.
no_complement = [st for st in only_one if len(st.split(' ')) ==1]

print(f'There are {len(no_complement)} entries in these condition.')

#Remove from only_one
only_one = [st for st in only_one if st not in no_complement]

There are 560 entries in these condition.


For each entry in no_complement we'll check which streets in the streetbook has a match, for example, we have "Washington" in no_complement, and in streetbook there are 'Mount Washington Place', 'Mount Washington Way', 'North Washington Street', 'Washington Park', 'Washington Place', 'Washington Street', 'Washington Street Place', so that fits our condition.

In [52]:
#Search for matches
matches = {}
for street in no_complement:
    matches[street] = [st for st in streetbook if street in st]

There are some entries that appear in every streetbook. We'll separate these streets in another variable. Then we use difflib to check for a match.

In [53]:
#Get only the streets that return no matches from streetbook
no_comp_missp = {key:value for key, value in matches.items() if len(value) == 0}

In [54]:
#Get just the entries that has a close match in streetbook
missp_clean = {key:value for key, value in no_comp_missp.items() if difflib.get_close_matches(key, streetbook, cutoff=0.7)}

#Get a list of these possible matches
for missp in missp_clean:
    if difflib.get_close_matches(missp, streetbook):
        missp_clean[missp] = difflib.get_close_matches(missp, streetbook)

There are very few streets that have a close match, so we'll just use the ones that are really close. For example, Children's is more likely to be Children's Way, but it could also refer to Boston Children's Hospital. So it will be set to Unknown like the other entries that have nothing close.

In [55]:
#Check the entries that has a match
missp_clean

{'Windslow': ['Winslow Road', 'Winslow Street', 'Winslow Square'],
 "Children'S": ["Children's Way", 'Child Street'],
 'Greenway': ['Fenway', 'Greenwood Park', 'Greenwich Park'],
 'Commericial': ['Commercial Wharf', 'Commercial Court', 'Commercial Street'],
 'Nighttingale': ['Nightingale Street'],
 'Castlecourt': ['Castle Court', 'Cutler Court', 'Newcastle Court'],
 'Comercial': ['Commercial Wharf', 'Commercial Court', 'Commercial Street'],
 'Allstae': ['Allstate Road', 'Allston Square'],
 'Mcgreevey': ['McGreevey Way'],
 'Northhampton': ['Northampton Street', 'Southampton Street'],
 'Washingtonst': ['Washington Street', 'Washington Park', 'Washington Mall'],
 'Dochester': ['Dorchester Bay', 'Dorchester Street', 'Dorchester Square']}

In [56]:
#Put the streets that has a close match into the correct variable
correct['Nighttingale'] = 'Nightingale Street'
correct['Castlecourt'] = 'Castle Court'
correct['Allstae'] = 'Allstate Road'
correct['Mcgreevey'] = 'McGreevey Way'
correct['Northhampton'] = 'Northhampton Street'
correct['Washingtonst'] = 'Washington Street'

Get all the streets that have only one match in the streetbook, set them to the correct variable and remove them from no_complement.

In [57]:
#Get all the streets that has just one possibilitie that fits our condition
one_correct = {key:value[0] for key, value in matches.items() if len(value)==1}

#Update the correct dictionary with the values above
correct.update({key:value for key, value in one_correct.items()})

#Remove from no_complement the streets that we put in the correct variable
no_complement = [st for st in no_complement if st not in correct]


#### <span style='color: #6fa832'>Only_one_clean</span>
We'll clean up the only_one_ variable using the remove_abb function, creating a new dictionary called only_one_clean.

In [58]:
#Replace some abbreviations 
only_one_clean = remove_abb(only_one)

In the dictionary we've created, there are some entries that contain ordinals, we'll replace them with writing. The entries in the dataset follow this pattern, so we keep it here too.

In [59]:
#Map to replace the ordinal values
ordinal_map = {'1St': 'First', '2Nd': 'Second', '3Rd': 'Third', '4Th': 'Fourth', '5Th': 'Fifth', '6Th': 'Sixth', '7Th': 'Seventh',
               '8Th': 'Eighth', '9Th': 'Ninth', '13Th': 'Thirteenth'}

In [60]:
#Remove the ordinals from the not_similar list
ordinals = ['1St', '2Nd', '3Rd', 'Th']
to_correct = []

#Make a list of the streets that have an ordinal number
for ord in ordinals:
    to_correct = to_correct + [st for st in only_one_clean if ord in st]

#Enter the correct street name in the dictionary correct
for st in to_correct:
    split = st.split('#')[0]
    for key in ordinal_map:
        if key in split:
            only_one_clean[st] = only_one_clean[st].replace(key, ordinal_map[key])

After the changes, we'll check again if some of the streets are already in the streetbook, set them to the correct dictionary and remove them from only_one_clean.

In [61]:
#Check if there is a street name in the streetbook after removing the abbreviation
comp_sb = compare_streetbook(list(only_one_clean.values()))
correct = {key: value for key, value in only_one_clean.items() if value not in comp_sb}
only_one_clean = {key:value for key, value in only_one_clean.items() if value in comp_sb}

In [62]:
#Length of each list
print(f'We have {len(two_streets)} points of intersection between two roads or streets.')
print(f'We have {len(only_one_clean)} individual streets that are not in the Boston streetbook.')

We have 630 points of intersection between two roads or streets.
We have 842 individual streets that are not in the Boston streetbook.


There are some roads in the only_one_clean list that have a different structure, we'll deal with those first.

In [63]:
#Get all the roads that have a different structure
different = [st for st in only_one_clean if '\n' in st]
print(f'There are {len(different)} streets that has a different structure.')

#Remove these streets from the only_one list
only_one = list(set(only_one).difference(different))

print(different)

There are 35 streets that has a different structure.
['133 Thornton\nRoxbury  Ma 02119\nUnited States', '40 Sudbury St\nBoston  Ma 02114\nUnited States', 'Granite Avenue\nDorchester  Ma 02124\nUnited States', '101 West Broadway\nBoston  Ma 02127\nUnited States', '531 Columbia Road\nDorchester  Ma 02125\nUnited Stat', '1 Schroeder Plaza\nBoston  Ma 02120\nUnited States', '1650 Ma-28\nBoston  Ma 02126\nUnited States', 'Dudley Station\nRoxbury  Ma 02119\nUnited States', '783 Blue Hill Avenue\nBoston  Ma 02124\nUnited State', '92 Olney St\nDorchester  Ma 02124\nUnited States', '42 Harrison Archway\nBoston  Ma 02118\nUnited States', '1165 Blue Hill Avenue\nBoston  Ma 02124\nUnited Stat', 'Island/Gerard\nRoxbury  Ma 02119\nUnited States', 'One Boston Medical Center Pl\nBoston  Ma 02118\nUnit', "136 Msgr O'Callaghan Way\nBoston  Ma 02127\nUnited S", '301 Washington St.\nBrighton  Ma. 02135\nUnited Stat', '2 Washington St\nHyde Park  Mass 02136\nUnited State', '24 Sudbury St\nBoston  Ma 02114\

Of these 35 streets, only "1650 Ma-28\nBoston Ma 02126\nUnited States" can't be found on the map. For the rest, we can find the exact location and the correct name of the street, so we put them in the appropriate json files.

In [64]:
#Removes the street from different variable
different.remove('1650 Ma-28\nBoston  Ma 02126\nUnited States')

Look at the only_one_clean and get the ones where key and value are the same. This may indicate that there are some streets that are already correct and are not in the streetbook.

In [65]:
#Get key and value that are equals
equals = {key: value for key, value in only_one_clean.items() if key==value}
print(f"We have {len(equals)} where the key is equal to the value.")

#Check for misspelled 
equals_misspelled = matches_search(list(equals.keys()))
print(f"There are {len(equals) - len(equals_misspelled)} misspelled streets.")

#Remaining Dictionary
equals = {key:value for key, value in equals.items() if key in equals_misspelled}

We have 463 where the key is equal to the value.
There are 170 misspelled streets.


After check the varbiable correct, there are some streets that need to be removed and others that need to be changed.

In [66]:
#Streets to remove
correct.pop("War Memorial")
correct.pop("North Washington Street Bridge")
correct.pop("No Street")

#Streets to change
correct['Boston University'] = "Bay State Road"
correct['Franklin Square Park'] = "Franklin Park Road"
correct['Kerr Way'] = 'Sojourner Truth Court'

In [67]:
#Remove the entries where key and value are equals
only_one_clean = {key:value for key, value in only_one_clean.items() if key not in equals}

The streets in only_one_clean that could somehow be identified were put into the correct_street json file.

#### <span style='color: #6fa832'>Working with the two_streets list</span>
Most of the two_streets list allows us to find the exact location. For these streets, we'll add them to the correct_street, location and suburb json files. 

In [68]:
#Streets that have no match on the map
two_streets_unknown = ['Albany St / South Station Ramp', 'Border And New', 'Cemetery Rd & Glen Rd\nBoston  Ma 02130\nUnited Stat', 'Clinton St & Commercial St\nBoston, Ma 02109\nUnited', 'Columbia Rd & General Lawrence J Logan Way\nBoston ', 'Commonwealth Ave & Essex St\nBoston  Ma 02116\nUnite', 'Grove St. @ Stimpson St.', 'Hampton St & Mass And Cass', 'Haymarket Sq & Surface St\nBoston  Ma 02109\nUnited ', 'High Street & Scanlon Drive\nRandolph  Ma 02368\nUni', 'Huntington Ave & Boylston St\nBoston, Ma 02116\nUnit', 'Main Street & Nonantum Road\nWatertown  Mass 02472\n', 'Maverick Sq & Meridean St\nE Boston  Ma 02128\nUnite', 'Morris St & Bennington St\nBoston  Ma 02128\nUnited ', 'Morton St & Jewish War Veterans Memorial Drive\nBos', 'N Harvard/Stadium Rd', 'Newmarket Sq & Allstate Rd\nBoston  Ma 02125\nUnited', 'Newmarket Sq & Allstate Rd\nBoston  Ma 02125\nUnited', 'Newmarket Sq & Allstate Rd\nBoston Ma 02125\nUnited', 'None & None\nBoston  Ma 02129\nUnited States', 'None & None\nBoston  Ma 02129\nUnited States', 'None & None\nBoston  Ma 02170\nUnited States', 'None & None\nBoston  Ma 02170\nUnited States', 'None & None\nBoston Ma 02129\nUnited States', 'Norfolk And Kemble', 'Old Atlantic Ave & Dead End\nBoston  Ma 02110\nUnite', 'Old Atlantic Ave & Dead End\nBoston  Ma 02110\nUnite', 'Old Atlantic Ave & Dead End\nBoston Ma 02110\nUnite', 'Old Colony / Mt. Vernon\nBoston  Ma. 02127\nUnited S', 'One Schroeder Plaza & Codmanhill Ave\nBoston  Massa', 'Reservation Road & Beacon Street\nBrighton  Ma 0213', 'Rotary Vfw Pkwy & W.R. Pkwy', 'S Between Exits 34 And 35', 'Sudbury Street & Blackstone Street\nBoston, Massach', 'Sullivan Sq/Alford St', 'Truman Pkwy And Blue Hill Ave', 'Truman Pkwy And Blue Hill Ave', 'Wilmot St & Clare Ave\nBoston  Ma 02136\nUnited Stat']

In [69]:
#Remove the above streets from the two_streets list
two_streets = list(set(two_streets).difference(two_streets_unknown))

#Check two_streets_variable
print(two_streets)

['Lucerne St & Floyd St\nBoston, Ma 02124\nUnited Stat', 'West Roxbury Pkwy & Weld St', 'Commercial St & Foster St\nBoston  Ma 02109\nUnited ', 'Ruggles St & Huntington Ave\nBoston  Ma 02116\nUnite', 'Edgewater Drive & River Street\nBoston  Ma 02126\nUn', 'Brookford St & Blue Hill Ave\nRoxbury  Ma 02119\nUni', 'Washington St & South St\nJamaica Plain  Ma 02130\nU', 'Wales St & Talbot Ave\nDorchester  Ma 02124\nUnited ', 'Boylston St And Dartmouth', 'Harvard St & Walk Hill St\nMattapan, Ma 02126\nUnite', 'Beacon St & Chestnut Hill Dr\nBrighton, Ma 02135\nUn', 'Boylston St & Exeter St\nBoston, Ma 02116\nUnited St', 'Albany St / Hampton St', 'Albany St & Paul Sullivan Way\nBoston  Ma 02118\nUni', 'Gallivan Blvd & Carruth St', 'Decatur Street At London Street', 'Ashmont St & Adams St\nDorchester, Ma 02124\nUnited ', 'Dale St & Rockland Ave\nRoxbury  Ma 02119\nUnited St', 'Albany & Melna Case', 'Clare Ave & American Legion Hwy\nHyde Park, Ma 0213', 'Shawmut Ave & Northampton St\nRoxbury  Ma

In [70]:
#Put all the streets in correct dictionary to correct_street json file
fill_json(correct)

#### <span style='color: #6fa832'> Import the json files

We have created 3 json files, one containing the location, suburb and correct name that we could find. We will use these files to replace the corresponding records in the dataset.

In [71]:
#Get the json file
json_location = read_json('../Json_Files/Location.json')
json_suburb = read_json('../Json_Files/Suburb.json')
json_correct = read_json('../Json_Files/correct_street.json')

We'll get all the indexes from the entries that are in the json_correct file to check after the changes.

In [72]:
#Get the entries in the dataset that are in json_correct to check the changes after
check_changes = boston[boston['STREET'].isin(json_correct)].index
print(f'In the correct_json file there are {len(check_changes)} from the dataset.')

In the correct_json file there are 26172 from the dataset.


Now we use the 3 json files we imported to change some of the street name, location and suburb entries in the dataset.

In [73]:
#Use Json to make changes to the Dataset
boston.loc[boston['Location'].isin(json_location), 'Location'] = boston['STREET'].map(json_location)
boston.loc[boston['Suburb'].isin(json_suburb), 'Suburb'] = boston['STREET'].map(json_suburb)
boston.loc[boston['STREET'].isin(json_correct), 'STREET'] = boston['STREET'].map(json_correct)

Check if there is a line where the latitude and longitude columns are filled in but the location column is not, or if the location column is filled in but not the other two.

In [74]:
#Check the dataset
print(boston[~(boston['Lat'].isna()) & (boston['Location'].isna())].shape[0])
print(boston[(boston['Lat'].isna()) & ~(boston['Location'].isna())].shape[0])

0
0


After all these changes it's assumed that all the streets that have been corrected are in the streetbook variable, so we set all those that are not as Unknown. As we saw above, there are no rows where we have filled the Location column but not the Latitude and Longitude, so we set all the rows where there is a missing value to Unknown. The same for the Suburb column.

In [75]:
#Set the rest as unknown
boston.loc[~(boston['STREET'].isin(streetbook)),'STREET'] = 'Unknown'
boston.loc[boston['Location'].isna(), ['Lat', 'Long', 'Location']] = 'Unknonw'
boston.loc[boston['Suburb'].isna(),'Suburb'] = 'Unknown'

#### <span style='color: #6fa832'>Missing Districts</span>
The last column to be filled in is the districts. We'll search the dataset if there's only one district for each street. The rest that cannot be found are also set to Unknown.

In [76]:
#Fill in the districts using the dataset
def get_district(streets:list) -> dict:
    dists = {}
    for street in streets:
        aux = boston[(boston['STREET'] == street) & (~boston['DISTRICT'].isna())]['DISTRICT'].unique()
        if len(aux) == 1:
            dists[street] = aux[0]
    
    return dists

In [77]:
#Use the function above to fill in the missing districts
boston.loc[(boston['STREET'] != 'Unknown') & (boston['DISTRICT'].isna()), 'DISTRICT'] \
    = boston['STREET'].map(get_district(boston[(boston['STREET'] != 'Unknown') & (~boston['DISTRICT'].isna())]['STREET'].unique()))

In [78]:
#Set the missing districts to Unknown
boston.loc[boston['DISTRICT'].isna(), 'DISTRICT'] = 'Unknown'

### <span style='color: #339966'> Final Adjustments </span>

Looking for some of the offense_code_group categories, 'Manslaughter' appears twice simply because there is a space at the end of the word. The offense_decription column also has one category that appears twice, but it is a misspelled word.
The reporting_area column is really hard to fill in the missing values, so we set them all to Unknown.
The last step is to save both boston crimes dataset and shooting in a csv file.

In [79]:
#Remove the space at the end of the word
boston.loc[boston['OFFENSE_CODE_GROUP'] == 'Manslaughter ', 'OFFENSE_CODE_GROUP'] = 'Manslaughter'
boston.loc[boston['OFFENSE_DESCRIPTION'] == 'Murder, Non-Negligient Manslaughter', 'OFFENSE_DESCRIPTION'] = \
        'Murder, Non-Negligent Manslaughter'

In [80]:
#Set all the missing in REPORTING_AREA as Unknown
boston.loc[boston['REPORTING_AREA'].isna(), 'REPORTING_AREA'] = 'Unknown'

In [81]:
#Saves the clean dataset to a csv file
boston.to_csv('../DataSets/boston_clean.csv', index=False)

shooting.to_csv('../DataSets/Shootings_changed.csv', index=False)